# <center>Designing & Creating a Normalized Database</center>
## <center>Major League Baseball Data from Retrosheet</center>

## Part I - Getting to Know the Data (EDA)

In [1]:
#1 - read files
import pandas as pd

game=pd.read_csv('game_log.csv')
park=pd.read_csv('park_codes.csv')
person=pd.read_csv('person_codes.csv')
team=pd.read_csv('team_codes.csv')

pd.set_option('max_columns',180)
pd.set_option('max_rows',200000)
pd.set_option('max_colwidth',5000)

/dataquest/system/env/python3/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (12,13,14,15,19,20,81,82,83,84,85,86,87,88,93,94,95,96,97,98,99,100,105,106,108,109,111,112,114,115,117,118,120,121,123,124,126,127,129,130,132,133,135,136,138,139,141,142,144,145,147,148,150,151,153,154,156,157,160) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
#1 - text file explaining fields from main file - game
!cat game_log_fields.txt

Field(s)  Meaning
    1     Date in the form "yyyymmdd"
    2     Number of game:
             "0" -- a single game
             "1" -- the first game of a double (or triple) header
                    including seperate admission doubleheaders
             "2" -- the second game of a double (or triple) header
                    including seperate admission doubleheaders
             "3" -- the third game of a triple-header
             "A" -- the first game of a double-header involving 3 teams
             "B" -- the second game of a double-header involving 3 teams
    3     Day of week  ("Sun","Mon","Tue","Wed","Thu","Fri","Sat")
  4-5     Visiting team and league
    6     Visiting team game number
          For this and the home team game number, ties are counted as
          games and suspended games are counted from the starting
          rather than the ending date.
  7-8     Home team and league
    9     Home team game number
10-11     Visiting and home team score (unquoted)


In [3]:
#1 - game_log explore data
print('game (r,c) ' + str(game.shape))
game.tail(10)

game (r,c) (171907, 161)


,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
171897,20161002,0,Sun,OAK,AL,162,SEA,AL,162,3,2,54.0,D,NaN,NaN,NaN,SEA03,24856.0,154.0,102000000,000020000,31.0,7.0,2.0,0.0,1.0,3.0,0.0,0.0,0.0,4.0,0.0,9.0,0.0,0.0,3.0,0.0,5.0,4.0,2.0,2.0,0.0,0.0,27.0,9.0,0.0,0.0,1.0,0.0,32.0,6.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,10.0,0.0,0.0,1.0,0.0,4.0,5.0,3.0,3.0,0.0,0.0,27.0,8.0,0.0,0.0,3.0,0.0,joycj901,Jim Joyce,fairc901,Chad Fairchild,hoyej901,James Hoye,hudsm901,Marvin Hudson,NaN,NaN,NaN,NaN,melvb001,Bob Melvin,servs002,Scott Servais,manas001,Sean Manaea,hernf002,Felix Hernandez,axfoj001,John Axford,vogts001,Stephen Vogt,manas001,Sean Manaea,hernf002,Felix Hernandez,semim001,Marcus Semien,6.0,wendj002,Joey Wendle,4.0,vogts001,Stephen Vogt,10.0,healr001,Ryon Healy,5.0,alony001,Yonder Alonso,3.0,maxwb001,Bruce Maxwell,2.0,eibnb001,Brett Eibner,8.0,olsom001,Matt Olson,9.0,smolj002,Jake Smolinski,7.0,hereg002,Guillermo Heredia,8.0,omals001,Shawn O'Malley,6.0,canor001,Robinson Cano,10.0,gutif001,Franklin Gutierrez,9.0,seagk001,Kyle Seager,5.0,lee-d004,Dae-Ho Lee,3.0,gameb001,Ben Gamel,7.0,iannc001,Chris Iannetta,2.0,freem002,Mike Freeman,4.0,NaN,Y
171898,20161002,0,Sun,TBA,AL,162,TEX,AL,162,6,4,60.0,D,NaN,NaN,NaN,ARL02,37015.0,240.0,0210001002,0101000110,42.0,13.0,6.0,0.0,0.0,5.0,0.0,1.0,0.0,3.0,0.0,8.0,0.0,1.0,1.0,0.0,10.0,7.0,4.0,4.0,0.0,0.0,30.0,4.0,0.0,0.0,0.0,0.0,40.0,11.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,6.0,0.0,14.0,2.0,0.0,0.0,0.0,12.0,7.0,5.0,5.0,4.0,0.0,30.0,15.0,2.0,0.0,1.0,0.0,dejer901,Ramon De Jesus,wintm901,Mike Winters,muchm901,Mike Muchlinski,wegnm901,Mark Wegner,NaN,NaN,NaN,NaN,cashk001,Kevin Cash,banij001

- This is the main data set and includes all the details of the game such as:
    - The date, location, time, duration
    - The stats about the team and players

<center>**Columns 106 - 132 visiting team defensive positions <br>
Columns 133 - 159 home team defensive positions <br>
Positions listed [(1-9)](https://en.wikipedia.org/wiki/Baseball_positions)**</center>


| Number | Position |
| --- | --- |
| 1 | pitcher |
| 2 | catcher |
| 3 | first base |
| 4 | second base |
| 5 | third base |
| 6 | shortstop |
| 7 | left field |
| 8 | center field |
| 9 | right field |

In [4]:
#same leagues for h_league
game['v_league'].value_counts()

NL    88866
AL    74713
AA     5039
FL     1243
PL      532
UA      428
Name: v_league, dtype: int64

<center>**Columns 5 visiting team league <br>
Columns 8 home team league <br>
[Full Names](https://en.wikipedia.org/wiki/Baseball_positions)**</center>

| Abbreviation | Full Name | League Type | Status | 
| --- | --- | --- | --- |
| NL | National League | Major | Active |
| AL | American League |Major | Active |
| AA | American Association |Minor | Active |
| FL | Florida Leagueino | Minor | Active |
| PL | Players' League | Major | Terminated |
| UA | Union Association | Major | Terminated |

In [5]:
#1 - park_codes explore data
print('park (r,c) ' + str(park.shape))
park.head()

park (r,c) (252, 9)


,park_id,name,aka,city,state,start,end,league,notes
0,ALB01,Riverside Park,NaN,Albany,NY,09/11/1880,05/30/1882,NL,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,NaN,Altoona,PA,04/30/1884,05/31/1884,UA,NaN
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,04/19/1966,NaN,AL,NaN
3,ARL01,Arlington Stadium,NaN,Arlington,TX,04/21/1972,10/03/1993,AL,NaN
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,04/11/1994,NaN,AL,NaN


- This data is about the name, location, league and duration of the park

In [6]:
#1 - person_codes explore data
print('person (r,c) ' + str(person.shape))
person.head()

person (r,c) (20494, 7)


,id,last,first,player_debut,mgr_debut,coach_debut,ump_debut
0,aardd001,Aardsma,David,04/06/2004,NaN,NaN,NaN
1,aaroh101,Aaron,Hank,04/13/1954,NaN,NaN,NaN
2,aarot101,Aaron,Tommie,04/10/1962,NaN,04/06/1979,NaN
3,aased001,Aase,Don,07/26/1977,NaN,NaN,NaN
4,abada001,Abad,Andy,09/10/2001,NaN,NaN,NaN


- This data is about the names, ID's and start date of people in the league such as players, manager, coaches and umpires

In [7]:
#1 - team_codes explore data
print('team (r,c) ' + str(team.shape))
ny_teams=team[team['city']=='New York']
ny_teams.head(10)

team (r,c) (150, 8)


,team_id,league,start,end,city,nickname,franch_id,seq
9,NYA,AL,1903,0,New York,Yankees,BLA,2
85,NY2,NaN,1871,1875,New York,Mutuals,NY2,1
86,NY3,NL,1876,1876,New York,Mutuals,NY2,2
87,NY4,AA,1883,1887,New York,Metropolitans,NY4,1
88,NYN,NL,1962,0,New York,Mets,NYN,1
89,NYP,PL,1890,1890,New York,Giants,NYP,1
131,NY1,NL,1883,1957,New York,Giants,TRN,2


- This data is about the teams in the league
    - If there is a 0 in the end column it means the team still exists

## Part II - Importing Data into SQLite

### Creating Command and Query Functions

In [17]:
import sqlite3

In [13]:
def run_command(c):
    with sqlite3.connect('mlb.db') as conn:
        conn.isolation_level = None
        conn.execute(c)

In [14]:
def run_query(q):
    with sqlite3.connect('mlb.db') as conn:
        return pd.read_sql(q,conn)

### Creating Tables in SQLite DataBase

In [22]:
#only run once
with sqlite3.connect('mlb.db') as conn:
    game.to_sql('game_log',conn,index=False)
    park.to_sql('park_codes',conn,index=False)
    person.to_sql('person_codes',conn,index=False)
    team.to_sql('team_codes',conn,index=False)

In [23]:
run_query('''SELECT * FROM game_log LIMIT 3''')

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,None,1,FW1,None,1,0,2,54.0,D,None,None,None,FOR01,200.0,120.0,000000000,010010000,30.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,None,6.0,1.0,None,-1.0,None,4.0,1.0,1.0,1.0,0.0,0.0,27.0,9.0,0.0,3.0,0.0,0.0,31.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,None,0.0,0.0,None,-1.0,None,3.0,1.0,0.0,0.0,0.0,0.0,27.0,3.0,3.0,1.0,1.0,0.0,boakj901,John Boake,None,None,None,None,None,None,None,None,None,None,paboc101,Charlie Pabor,lennb101,Bill Lennon,mathb101,Bobby Mathews,prata101,Al Pratt,None,None,None,None,prata101,Al Pratt,mathb101,Bobby Mathews,whitd102,Deacon White,2.0,kimbg101,Gene Kimball,4.0,paboc101,Charlie Pabor,7.0,allia101,Art Allison,8.0,white104,Elmer White,9.0,prata101,Al Pratt,1.0,sutte101,Ezra Sutton,5.0,carlj102,Jim Carleton,3.0,bassj101,John Bass,6.0,selmf101,Frank Sellman,5.0,mathb101,Bobby Mathews,1.0,foraj101,Jim Foran,3.0,goldw101,Wally Goldsmith,6.0,lennb101,Bill Lennon,2.0,caret101,Tom Carey,4.0,mince101,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,None,Y
1,18710505,0,Fri,BS1,None,1,WS3,None,1,20,18,54.0,D,None,None,None,WAS01,5000.0,145.0,107000435,640113030,41.0,13.0,1.0,2.0,0.0,13.0,0.0,0.0,0.0,18.0,None,5.0,3.0,None,-1.0,None,12.0,1.0,6.0,6.0,1.0,0.0,27.0,13.0,10.0,1.0,2.0,0.0,49.0,14.0,2.0,0.0,0.0,11.0,0.0,0.0,0.0,10.0,None,2.0,1.0,None,-1.0,None,14.0,1.0,7.0,7.0,0.0,0.0,27.0,20.0,10.0,2.0,3.0,0.0,dobsh901,Henry Dobson,None,None,None,None,None,None,None,None,None,None,wrigh101,Harry Wright,younn801,Nick Young,spala101,Al Spalding,braia102,Asa Brainard,None,None,None,None,spala101,Al Spalding,braia102,As

### Craeting New Columns in Tables

In [ ]:
c_newcol=
ALTER TABLE game_log ADD COLUMN game_id
UPDATE game_log SET game_id = date+h_name+number_of_game